#### Librerias

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from time import sleep
import csv
import pandas as pd

#### Mostramos la cantidad de comentarios que hay en la pagina principal

In [9]:
# Inicia el WebDriver
driver = webdriver.Chrome()
url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)
sleep(5)

try:
    review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]') #article[aria-roledescription="Review"], p[role="presentation"]
    
    # Verificá cuántos artículos se encontraron
    print(f"Se encontraron {len(review_articles)} artículos de reseña.")
    
except Exception as e:
    print(f"No se pudieron encontrar artículos de reseñas: {e}")

driver.quit()
# input("Presiona Enter para salir...")


Se encontraron 15 artículos de reseña.


#### Mostramos la cantidad de comentarios que hay en la pagina principal haciendo Scroll

In [5]:

# Inicia el WebDriver
driver = webdriver.Chrome()

url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)

sleep(5) 

# Número total de reseñas
total_reviews = 0
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Busca los comentarios
    review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
    total_reviews = len(review_articles)
    print(f"Total de reseñas encontradas: {total_reviews}")

    # Hace el scroll en la web
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    sleep(2)

    # Verificá si llegó al final de la página
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

print(f"Total final de reseñas: {total_reviews}")
driver.quit()
# input("Presiona Enter para salir...")

Total de reseñas encontradas: 15
Total de reseñas encontradas: 30
Total de reseñas encontradas: 45
Total de reseñas encontradas: 60
Total de reseñas encontradas: 75
Total de reseñas encontradas: 90
Total de reseñas encontradas: 105
Total de reseñas encontradas: 120
Total de reseñas encontradas: 135
Total de reseñas encontradas: 150
Total de reseñas encontradas: 165
Total de reseñas encontradas: 180
Total de reseñas encontradas: 195
Total de reseñas encontradas: 210
Total final de reseñas: 210


''

## Desarrolló
Logramos ver que los comentarios que aparecian en la pagina principal no eran absolutamente todos, así que decidimos hacer un scraping por la categoria de comentarios, es decir, un scraping que comience por la categoria de 5 estrellas hasta la categoria de 1 una estrella, por eso comenzaremos con la interacción del botón de catacteristicas

#### Comenzamos la interacción con el botón "Calificaciones"

In [7]:
driver = webdriver.Chrome()
url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)

try:
    mostrar_mas_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]') #//button[@data-testid='see-more']    
    mostrar_mas_button.click()
    print("Botón de 'Mostrar más comentarios' fue clickeado correctamente.")
    (5)
    
except Exception as e:
    print(f"No se pudo hacer clic en el botón: {e}")
    driver.quit()
    exit()

driver.quit()
# input("Presiona Enter para salir...")

Botón de 'Mostrar más comentarios' fue clickeado correctamente.


''

In [8]:
driver = webdriver.Chrome()
url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)
sleep(3)

ratings = ['5', '4', '3', '2', '1']           # Las categorias usan el mismo XPATH, lo unico que cambia es el numero
estrellas = ['1', '2', '3', '4', '5']         #Para que no se confunda las categorias

# Variable acumula para el total de reseñas
total_comments = 0

for rating, estrella in zip(ratings, estrellas):
    while True: 
        rating_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]')
        rating_button.click()
        sleep(3)

        
        try:
            rating_option = WebDriverWait(driver, 3).until(
                EC.visibility_of_element_located((By.XPATH, f'//li[@data-testid="filterItem-rating-{rating}"]'))
            )
            rating_option.click()
            sleep(3)  


            review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
            total_reviews = len(review_articles)
            print(f"Total de reseñas con {estrella} estrella(s): {total_reviews}")

            # Suma de receñas
            total_comments += total_reviews

            break 

        except TimeoutException:
            print(f"No se pudo encontrar la opción de {rating} estrella(s).")
            break 

# Imprimir el total de reseñas
print(f"Total de reseñas en todas las categorías: {total_comments}")

driver.quit()

Total de reseñas con 1 estrella(s): 15
Total de reseñas con 2 estrella(s): 15
Total de reseñas con 3 estrella(s): 15
Total de reseñas con 4 estrella(s): 15
Total de reseñas con 5 estrella(s): 15
Total de reseñas en todas las categorías: 75


In [9]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

# Navegar a la página de producto en Mercado Libre
url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)
sleep(5)


ratings = ['5', '4', '3', '2', '1']     # Las categorias usan el mismo XPATH, lo unico que cambia es el numero
estrellas = ['1', '2', '3', '4', '5']   #Para que no se confunda las categorias
total_comments = 0

# Recorrer cada calificación
for rating, estrella in zip(ratings, estrellas):
    while True:
        rating_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]')
        rating_button.click()
        sleep(5)

        # Elije la clasificación, de 1 a 5 estrellas
        try:
            rating_option = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, f'//li[@data-testid="filterItem-rating-{rating}"]'))
            )
            rating_option.click()
            sleep(5) 

            # Bucle para hacer scroll hasta el final y contar los comentarios
            total_reviews = 0
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                # Hace el scroll
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(2)

                # Verifica si llega al final de la página
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Cuenta comentarios al final del scroll
            review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
            total_reviews = len(review_articles)
            print(f"Total de reseñas con {estrella} estrella(s): {total_reviews}")

            # Suma de receñas
            total_comments += total_reviews

            # Volver al inicio
            driver.execute_script("window.scrollTo(0, 0);")
            sleep(5) 

            break 

        except TimeoutException:
            print(f"No se pudo encontrar la opción de {estrella} estrella(s).")
            break  

# Imprimir el total de reseñas
print(f"Total de reseñas en todas las categorías: {total_comments}")
driver.quit()

Total de reseñas con 1 estrella(s): 125
Total de reseñas con 2 estrella(s): 66
Total de reseñas con 3 estrella(s): 176
Total de reseñas con 4 estrella(s): 210
Total de reseñas con 5 estrella(s): 210
Total de reseñas en todas las categorías: 787


In [4]:
# Inicializar el WebDriver
driver = webdriver.Chrome()

# Navegar a la página de producto en Mercado Libre
url = 'https://www.mercadolibre.com.ar/noindex/catalog/reviews/MLA18955016?noIndex=true&access=view_all&modal=true&controlled=true'
driver.get(url)
sleep(5)

ratings = ['5', '4', '3', '2', '1']  # Categorías (XPATH cambia solo el número)
estrellas = ['5', '4', '3', '2', '1']  # Para evitar confusión entre categorías

# Lista para almacenar los datos de los comentarios
comentarios = []

# Recorrer cada calificación
for rating, estrella in zip(ratings, estrellas):
    while True:
        rating_button = driver.find_element(By.XPATH, '//button[@id=":R1inbh:"]')
        rating_button.click()
        sleep(5)

        # Selecciona la calificación de 1 a 5 estrellas
        try:
            rating_option = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, f'//li[@data-testid="filterItem-rating-{rating}"]'))
            )
            rating_option.click()
            sleep(5)

            # Bucle para hacer scroll hasta el final
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                # Hace scroll
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                sleep(2)

                # Verifica si llega al final de la página
                new_height = driver.execute_script("return document.body.scrollHeight")
                if new_height == last_height:
                    break
                last_height = new_height

            # Extraer datos de los comentarios
            review_articles = driver.find_elements(By.CSS_SELECTOR, 'article[aria-roledescription="Review"]')
            for review in review_articles:
                try:
                    fecha = review.find_element(By.CSS_SELECTOR, 'span.ui-review-capability-comments__comment__date').text
                    comentario = review.find_element(By.CSS_SELECTOR, 'p[data-testid="comment-content-component"]').text
                    likes = review.find_element(By.CSS_SELECTOR, 'p.ui-review-capability-valorizations__button-like__text').text
                    estrellas_comentario = review.find_element(By.CSS_SELECTOR, 'div.ui-review-capability-comments__comment__rating').text

                    # Guardar los datos en la lista
                    comentarios.append([fecha, comentario, likes, estrellas_comentario])
                except Exception as e:
                    print(f"Error al extraer datos de un comentario: {e}")

            # Volver al inicio
            driver.execute_script("window.scrollTo(0, 0);")
            sleep(5)

            break

        except Exception as e:
            print(f"No se pudo encontrar la opción de {estrella} estrella(s). Error: {e}")
            break

# Guardar los datos en un archivo CSV
with open('comentarios_SA03.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Fecha', 'Comentario', 'Estrellas', 'Likes'])
    writer.writerows(comentarios)

# Cerrar el navegador
driver.quit()

In [10]:
df = pd.read_csv('comentarios_SA03.csv')

In [11]:
df.head()

,Fecha,Comentario,Estrellas,Likes
0,15 ago. 2022,"En mi opinión anda un 50 del 100, se traba y s...",5,Calificación 1 de 5
1,10 jun. 2023,"Malo, muy malo, tenemos dos y los dos salieron...",4,Calificación 1 de 5
2,02 jun. 2023,"Una decepción es este teléfono, te pones a mir...",4,Calificación 1 de 5
3,02 nov. 2023,Falla el táctil.,3,Calificación 1 de 5
4,07 oct. 2022,Necesito que me lo cambien. Hoy se apagó y no ...,3,Calificación 1 de 5
